This is the open source code of paper: Utilizing Domain Knowledge: Robust Machine Learning for Building Energy Prediction with Small, Inconsistent Datasets.
- @Date : 2022-06-24 16:21:35
- @Link : https://arxiv.org/abs/2302.10784
- @Ver : v02
- @Author: Xia CHEN (xia.chen@iek.uni-hannover.de), Xia Chen, Manav Mahan Singh, Philipp Geyer
---
For using the code or data, please cite:

*- Chen, X., Singh, M.M. and Geyer, P., 2023. Utilizing Domain Knowledge: Robust Machine Learning for Building Energy Prediction with Small, Inconsistent Datasets. arXiv preprint arXiv:2302.10784.*

*- CHEN, Xia; Singh, Manav Mahan; Geyer, Philipp (2023), “Utilizing domain knowledge: robust machine learning for building energy performance prediction with small, inconsistent datasets”, Mendeley Data, V2, doi: 10.17632/fctghwx3r9.2*

---
This code is designed to preprocess and transform building energy data for different components and zones for specific typical days, focusing on the winter period. The goal is to prepare the data for machine learning models, whether for training or testing purposes. Here’s a detailed breakdown of the script’s functionality:

---
1. **Define Time Period and Component Files**: It starts by specifying the analysis period (`'Winter Typical'`) and initializing the `COMPONENT` variable to the first file in a list of files. The script iterates over each file in this list, identifying the type of component (building, zone, etc.) based on the file name.
---

2. **Select Target Variables**: For each component type, it selects different target variables relevant to the analysis. For example, if the file pertains to typical days in a building, it focuses on the heating load. If it’s about zone typical\ days, it considers various heat flows and the heating load as targets.
---

3. **Data Loading and Filtering**: The script differentiates between training and testing scenarios using the `TRAIN` boolean flag. It loads the appropriate dataset and, if a fast processing mode (`FAST`) is selected, samples a fraction of the data to expedite the analysis. This step is crucial for handling large datasets efficiently.
---

4. **Data Preparation**:
    - **Column Renaming**: Columns prefixed with 'Building:' are renamed to 'Building_' to standardize column names.
    - **Target Column Identification**: It identifies columns related to the specified period and prepares them for extraction and analysis.
    - **Data Expansion**: For files with multiple values in a single cell (separated by semicolons), it expands these values into separate rows or columns, depending on the target variable. This is particularly important for the zone data, where heat flows are detailed.

---

5. **Output Dataframe Construction**: It constructs an output dataframe that:
    - Excludes unwanted columns.
    - Ensures the data is in a format suitable for machine learning models, converting string values to floats and rounding them for consistency.
    - Joins weather data to the main dataframe, repeating weather entries to match the length of the main dataframe, ensuring each entry has corresponding weather information.
---

6. **Memory Optimization**: The script employs a function to reduce the memory usage of the dataframe, an essential step for handling large datasets efficiently.

---
7. **Saving the Transformed Data**: Finally, it saves the prepared dataframe to a CSV file, naming the file according to the component and the period analyzed. This naming convention facilitates easy identification of the dataset's contents.


In [1]:
import pandas as pd
import numpy as np
import os, sys, gc, time, warnings, pickle, psutil, random
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn import metrics
import csv
from math import ceil

from tqdm import tqdm

import plotly.offline as py                    
py.init_notebook_mode(connected=True)          
import plotly.graph_objs as go                 
import plotly.figure_factory as ff             

# Target plot
import seaborn as sns, matplotlib.pyplot as plt
import random 

# For model training
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# pd.set_option('max_columns', 100)

warnings.filterwarnings('ignore')
ROUTE = '../BoxTrainData/'
ROUTE_test = '../RepTestData/'

PERIOD = 'Winter Typical'
TRAIN = True
# Sample
FAST = True
# Traverse files
g = os.walk(ROUTE)  

files = []
for path,dir_list,file_list in g:  
    for file_name in file_list:  
        files.append(file_name)
files = files[1:]

In [2]:
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
# Merge with weather data

weather_df = pd.read_csv('../Weather_data.csv')
# to date_time and set as index
date_df = weather_df[['Year', 'Month', 'Day', 'Hour']]
date = pd.to_datetime(date_df)

weather_df.index = date

del date_df, date

weather_df = weather_df[['Year', 'Month', 'Day', 'Hour','Temp','Dew','Hum','Pres']]
weather_df

,Year,Month,Day,Hour,Temp,Dew,Hum,Pres
1986-01-01 01:00:00,1986,1,1,1,-7.2,-8.4,90,95300
1986-01-01 02:00:00,1986,1,1,2,-6.4,-7.6,90,94900
1986-01-01 03:00:00,1986,1,1,3,-5.9,-7.1,90,94600
1986-01-01 04:00:00,1986,1,1,4,-5.6,-6.8,90,94500
1986-01-01 05:00:00,1986,1,1,5,-5.5,-6.6,91,94500
...,...,...,...,...,...,...,...,...
1984-12-31 20:00:00,1984,12,31,20,-10.6,-11.1,96,96900
1984-12-31 21:00:00,1984,12,31,21,-10.4,-10.9,95,96800
1984-12-31 22:00:00,1984,12,31,22,-9.9,-10.6,94,96600
1984-12-31 23:00:00,1984,12,31,23,-9.1,-10.0,92,96200


In [4]:
# Select the corresponding weather date
if(PERIOD == 'Winter Typical'):
    # 1/5-1/17
    target_weather = weather_df.loc[(weather_df['Month'] == 1)&(weather_df['Day'] >= 5)&(weather_df['Day'] <= 17)]
if(PERIOD == 'Winter Extreme'):
    # 2/9-2/21
    target_weather = weather_df.loc[(weather_df['Month'] == 2)&(weather_df['Day'] >= 9)&(weather_df['Day'] <= 21)]
if(PERIOD == 'Spring'):
    # 3/28-4/10
    target_weather = weather_df.loc[(weather_df['Month'] == 2)&(weather_df['Day'] >= 9)&(weather_df['Day'] <= 21)]
if(PERIOD == 'Summer Typical'):
    # 7/12 - 7/24
    target_weather = weather_df.loc[(weather_df['Month'] == 7)&(weather_df['Day'] >= 12)&(weather_df['Day'] <= 24)]
if(PERIOD == 'Summer Extreme'):
    # 7/19-7/31
    target_weather = weather_df.loc[(weather_df['Month'] == 7)&(weather_df['Day'] >= 19)&(weather_df['Day'] <= 31)]
if(PERIOD == 'Autumn'):
    # 10/19-10/31
    target_weather = weather_df.loc[(weather_df['Month'] == 10)&(weather_df['Day'] >= 19)&(weather_df['Day'] <= 31)]
target_weather

,Year,Month,Day,Hour,Temp,Dew,Hum,Pres
1986-01-05 01:00:00,1986,1,5,1,-5.4,-7.7,82,95700
1986-01-05 02:00:00,1986,1,5,2,-6.9,-8.4,88,95700
1986-01-05 03:00:00,1986,1,5,3,-6.6,-7.9,89,95700
1986-01-05 04:00:00,1986,1,5,4,-8.2,-10.7,80,95600
1986-01-05 05:00:00,1986,1,5,5,-7.9,-10.2,82,95600
...,...,...,...,...,...,...,...,...
1986-01-17 20:00:00,1986,1,17,20,-2.1,-2.7,95,96300
1986-01-17 21:00:00,1986,1,17,21,-1.9,-2.9,92,96400
1986-01-17 22:00:00,1986,1,17,22,-4.2,-5.0,93,96500
1986-01-17 23:00:00,1986,1,17,23,-4.9,-5.5,95,96600


In [5]:
# PERIOD = 'Winter Typical'

# for
# building, wall, zone...
COMPONENT = files[0]
for COMPONENT in files:
    if(COMPONENT == 'BuildingTypicalDays.csv'):
        TARGET = PERIOD+':Heating Load'
    elif(COMPONENT == 'ZoneTypicalDays.csv'):
        TARGET = [PERIOD+':WallWindow Heat Flow',
                 PERIOD+':GFloor Heat Flow',
                 PERIOD+':Roof Heat Flow',
                 PERIOD+':Infiltration Heat Flow',
                 PERIOD+':Heating Load']
    else:
        TARGET = PERIOD+':Heat Flow'
    if(TRAIN == True):
        # 读文件
        target_df = pd.read_csv(ROUTE+COMPONENT)
        print('Training Data')
        if(FAST is True):
            if(COMPONENT == 'BuildingTypicalDays.csv'):
                #### SAMPLE!
                TARGET_BUILDING = target_df['File'].sample(frac=0.000125)
#                 TARGET_BUILDING = target_df['File'][:1000]
                target_df = target_df[target_df['File'].isin(TARGET_BUILDING.values.tolist())]
                print(TARGET_BUILDING)
            else:
                target_df = target_df[target_df['File'].isin(TARGET_BUILDING.values.tolist())]
        elif(FAST is False):
            TARGET_BUILDING = target_df['File']
            target_df = target_df[target_df['File'].isin(TARGET_BUILDING.values.tolist())]
    else:
        target_df = pd.read_csv(ROUTE_test+COMPONENT)
        print('Testing Data')
        if(FAST is True):
            if(COMPONENT == 'BuildingTypicalDays.csv'):
            
#                 TARGET_BUILDING = target_df['File'][:300]
                TARGET_BUILDING = target_df['File']
                target_df = target_df[target_df['File'].isin(TARGET_BUILDING.values.tolist())]
                print(TARGET_BUILDING)
            else:
                target_df = target_df[target_df['File'].isin(TARGET_BUILDING.values.tolist())]
        elif(FAST is False):
            TARGET_BUILDING = target_df['File']
            target_df = target_df[target_df['File'].isin(TARGET_BUILDING.values.tolist())]
            
    target_df.columns = target_df.columns.str.replace('Building:', 'Building_') 

    melt_target = [x for x in target_df.columns if PERIOD in x]
    out_target = [x for x in target_df.columns if ':' in x]

    retD = list(set(target_df.columns) - set(out_target))
    retD.sort(key=list(target_df.columns).index)

    output_df = target_df[retD]
    print('-'*50)
    print(COMPONENT)
    print(TARGET)
    print(output_df.shape)

    num_dict = pd.DataFrame(target_df['File'].value_counts())
    # num_dict['name'] = num_dict.index
    num_dict = num_dict.loc[target_df['File'].drop_duplicates().tolist()]
    num_dict
    
    if(COMPONENT == 'ZoneTypicalDays.csv'):
        for each_target_column in TARGET:
            print(each_target_column)

            indicator = each_target_column.replace(PERIOD,'')
            indicator = indicator.replace(':','')
            indicator = indicator.replace(' Heat Flow','')

            if(indicator == 'GFloor' or indicator == 'Roof'):
                print(indicator)
                test_df = target_df[['File', each_target_column]].dropna()

                # New Dataframe
                new_df = pd.DataFrame()
                for a,b in zip(num_dict.index.tolist(), num_dict['File']):
    #                 print(a,b)
                    for i in range(0,b):
                        new_df = pd.concat([new_df,test_df[test_df['File'] == a]],axis=0)
                expand_df = new_df[each_target_column].str.split(';',expand=True).stack()
                expand_df = expand_df.reset_index(level=1,drop=True).rename(each_target_column)
                print(expand_df.shape)
            else:
                expand_df = target_df[each_target_column].str.split(';',expand=True).stack()
                expand_df = expand_df.reset_index(level=1,drop=True).rename(each_target_column)
                print(expand_df.shape)

            if(each_target_column == TARGET[0]):
                output_df = output_df.join(expand_df)
            else:
                print(expand_df.head())
                output_df[each_target_column] = expand_df.values
            output_df[each_target_column] = output_df[each_target_column].astype(float)
            output_df = output_df.round(3)

    else:
        print('Not expand')
        for each_target_column in [TARGET]:
            print(each_target_column)
            expand_df = target_df[each_target_column].str.split(';',expand=True).stack()
            expand_df = expand_df.reset_index(level=1,drop=True).rename(each_target_column)
            print(expand_df.shape)

            output_df = output_df.join(expand_df)
            output_df[each_target_column] = output_df[each_target_column].astype(float)
            output_df = output_df.round(3)

    # merge target_weather
    target_weather = target_weather.reset_index(drop=True)
    repeat_time = int(len(output_df)/len(target_weather))

    merge_weather = pd.concat([target_weather]*repeat_time, ignore_index=True)
    output_df = output_df.reset_index(drop=True)
    output_df = pd.concat([merge_weather, output_df], axis=1)
    
    output_df = reduce_mem_usage(output_df)
    print('*'*25)
    print(output_df.shape)
    
    output_name =COMPONENT[:-4] + '_'+ PERIOD + '.csv'
    print(output_name)
    # file name: component + outputTARGET
    output_name = output_name.replace(':',' ')
    output_df.to_csv(output_name)


Training Data
369    Shape1_2291
Name: File, dtype: object
--------------------------------------------------
BuildingTypicalDays.csv
Winter Typical:Heating Load
(1, 35)
Not expand
Winter Typical:Heating Load
(312,)
Mem. usage decreased to  0.03 Mb (74.1% reduction)
*************************
(312, 44)
BuildingTypicalDays_Winter Typical.csv
Training Data
--------------------------------------------------
GFloorTypicalDays.csv
Winter Typical:Heat Flow
(1, 6)
Not expand
Winter Typical:Heat Flow
(312,)
Mem. usage decreased to  0.01 Mb (59.8% reduction)
*************************
(312, 15)
GFloorTypicalDays_Winter Typical.csv
Training Data
--------------------------------------------------
InfiltrationTypicalDays.csv
Winter Typical:Heat Flow
(3, 12)
Not expand
Winter Typical:Heat Flow
(936,)
Mem. usage decreased to  0.05 Mb (64.2% reduction)
*************************
(936, 21)
InfiltrationTypicalDays_Winter Typical.csv
Training Data
--------------------------------------------------
RoofTypi